# Smart-G demo notebook

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
from __future__ import absolute_import, division, print_function
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg
from smartg.smartg import RoughSurface, LambSurface, FlatSurface, Environment, Albedo_cst
from smartg.atmosphere import AtmAFGL, AeroOPAC, CloudOPAC, diff1, read_phase
from smartg.water import IOP_1, IOP
from smartg.reptran import REPTRAN, reduce_reptran
from smartg.tools.tools import SpherIrr, Irr, reduce_Irr
from smartg.tools.luts import LUT, MLUT, Idx, merge, read_mlut
from smartg.tools.smartg_view import plot_polar, smartg_view, input_view, mdesc, transect_view, profile_view, phase_view
from ipywidgets import interact, interact_manual
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))

# Quick Start

## Run your first simulation

In [ ]:
# run SMART-G
# this simulation uses an AFGL tropical atmosphere without aerosols (and absorption by Ozone and NO2)
# and a wind roughened sea surface
# Reflectance computation in Cone Sampling mode (default)
m = Smartg().run(THVDEG=30, wl=500., NBPHOTONS=1e8,
           atm=AtmAFGL('afglt'),
           surf=RoughSurface())

In [ ]:
# the result is a MLUT
m.describe(show_attrs=True)
# it can be written with m.save(filename)
# and then reloaded with read_mlut(filename)

## How to view the results

In [ ]:
# basic visualization
fig = smartg_view(m)

In [ ]:
# access a LUT within the MLUT
I = m['I_up (TOA)']  # or by index: m[0]
I.desc = mdesc(I.desc) # I descriptor in math Latex format

In [ ]:
I.print_info()

In [ ]:
# access values in this LUT
print(I[5, ::5])  # by index (can be float indices to interpolate)
print(I[Idx(90.), Idx(45.)])  # or by index finding + interpolation

In [ ]:
# calculate polarized light
# use operations between LUT's, and apply sqrt
Q, U = m['Q_up (TOA)'], m['U_up (TOA)']
LP = (Q*Q + U*U).apply(sqrt, mdesc('Lp_up (TOA)'))   # second parameter of apply sets the LUT's desc

In [ ]:
#Different visualizations
LP.plot_semi()
LP.plot()
LP.plot_polar()

In [ ]:
I.plot_semi(index=Idx([45]), vmin=0, vmax=0.3)   # the arguments are optional

In [ ]:
# get a sub-LUT from a LUT (here 2D -> 1D)
I.sub()[Idx(90),:].plot()

## A few examples

In [ ]:
# Basic Rayleigh example, no surface, full view
fig= smartg_view(Smartg(rng='CURAND_PHILOX').run(wl=500., THVDEG=30., NBPHOTONS=1e8, atm=AtmAFGL('afglt')), full=True)

In [ ]:
# only Rayleigh with a custom grid, a customes depolarization ratio and a custom surface pressure
fig= smartg_view(Smartg().run(wl=500., THVDEG=30., DEPO=0., NBPHOTONS=1e8,
        atm=AtmAFGL('afglt', grid='100[75]25[5]10[0.5]0.5', P0=990.)))

In [ ]:
# Rayleigh + aerosols (UV), downward radiance at the ground
aer = AeroOPAC('urban', 0.4, 550.)
pro = AtmAFGL('afglms', comp=[aer])
fig= smartg_view(Smartg().run(wl=322., THVDEG=60., atm=pro, NBPHOTONS=1e8, OUTPUT_LAYERS=3), field='down (0+)')

In [ ]:
# atmosphere + surface
wl= 490.
azimuth_transect = 10.
fig= smartg_view(Smartg().run(wl, NBPHOTONS=1e8, THVDEG=45.,
        atm=pro,
        surf=LambSurface(ALB=0.1)), ind=Idx([azimuth_transect]), QU=True)

In [ ]:
# atmosphere + surface + océan
atm   = AtmAFGL('afglms', O3=0., NO2=False)
surf  = RoughSurface(WIND=5., NH2O=1.34)
# Case I water Inherent Optical Properties depending on Chlorophyll concentration only
water = IOP_1(chl=0.5, DEPTH=5.)
# compute outputs at the surface and bottom of ocean also, 
# view results for upwelling and downwelling radiances
wl = 500.
th0= 60.
m=Smartg().run(wl, NBPHOTONS=1e8, THVDEG=th0, OUTPUT_LAYERS=3, BEER=1,
                atm=atm,  surf=surf, water=water)
fields=['up (TOA)','up (0+)','up (0-)','down (0+)','down (0-)','down (B)']
logIs =[True]*6
for logI,field in zip(logIs,fields):
        fig= smartg_view(m, field=field, logI=logI)

In [ ]:
# surface + ocean
atm   = None
surf  = RoughSurface(WIND=5., NH2O=1.34)
water = IOP_1(chl=0.5, DEPTH=5.)
# compute outputs at the surface also, view results for downwelling at bottom of ocean
fig= smartg_view(Smartg().run(wl, NBPHOTONS=1e8, THVDEG=th0, OUTPUT_LAYERS=3, BEER=1,
            atm=None,  surf=surf, water=water), 
            field='down (B)', logI=True)

In [ ]:
# ocean only
atm   = None
surf  = None
water = IOP_1(chl=0.5, DEPTH=5.)
# compute outputs at the surface also, view results for downwelling at bottom of ocean
fig= smartg_view(Smartg().run(wl, NBPHOTONS=1e8, THVDEG=th0, OUTPUT_LAYERS=3, BEER=1,
            atm=None,  surf=surf, water=water), 
            field='down (B)', logI=True)

# Advanced use

## Local Estimate

In [ ]:
%%time
# In order to compute accurately radiances in particular directions, set the LE mode
# 'le' keyword is a dictionnary containig directions vectors in radians and coded as float32
le={}
le.update(th = np.linspace(0,89., num=12)*np.pi/180)
le.update(phi= np.linspace(0,360, num=19)*np.pi/180)
# revisit atmosphere + surface + ocean
# the number of photons should be dramatically reduces since
# each photon participates to the computation of all directions
wl = 500.
th0= 60.
m = Smartg().run(wl=wl, NBPHOTONS=1e6, THVDEG=th0, OUTPUT_LAYERS=3, le=le,
            atm   = AtmAFGL('afglms', O3=0., NO2=False), 
            surf  = RoughSurface(WIND=5., NH2O=1.34),
            water=IOP_1(chl=0.5, DEPTH=5.))

fig= smartg_view(m, field='up (0-)', logI=True)

## Atmospheric profile
Aerosols and Cloud optical properties are those of OPAC. The files containing these properties have been
generated by www.libradtran.org. <br>
For the moment Only spherical particles are handled

In [ ]:
# monochromatic computation for custom aerosols and cloud
wl = 550.
# Aerosols and cloud optical properties using OPAC database as processed by the the libradtran (www.libradtran.org)
aer = AeroOPAC( 'desert', 0.001, wl) # set AOT at the reference wavelength wl to 0.3 
                                # and set aerosol type to 'desert'
cld = CloudOPAC('wc.sol', 11., 5., 10., 1., wl) # set cloud to water cloud with reff=11 mic.  
                                # the cloud is located between 2 and 4 km, with
                                # Optical thickness at the reference wavelength wl set to 1. 
                                                              
pro = AtmAFGL('afglt',    # tropical atmosphere
              comp=[aer, cld], # particles in atmosphere are a mix of aerosols and cloud            
              O3 = 0.,   # scale ozone vertical column to 0 Dobson units (here no absoprtion by ozone)
              NO2= False,# disable absorption by NO2
              H2O= 2.,   # scale water vapour column to 2 g/cm-2
              P0 = 980., # set sea level pressure to 980 hPa
              tauR=0.1,   # force Rayleigh optical thickness
              grid = [100, 75, 50, 30, 20, 10, 5, 2., 1.15], # set vertical grid, surface altitude at 1.15 km
              #pfgrid = [100., 10., 3., 1.15] # vertical grid for the computation of particles phase functions
              # here 3 phases functions between 1.15 and 3, 3 and 10, and 10 to TOA
              pfgrid=[100, 50, 30, 20, 10, 5, 2., 1.15]
             )

azimuth_transect = 30. 
m = Smartg().run(wl=wl, THVDEG=60., atm=pro, NBPHOTONS=1e8)
_ = smartg_view(m, ind=Idx([azimuth_transect]), logI=True)

In [ ]:
# View inputs
input_view(m, kind='atm')

In [ ]:
# Full aerosol customization
#1) import aerosols scattering matrix from external text file with 5 columns:
# angle, P11, P12, P33, P43
pha=read_phase('/home/did/RTC/SMART-G/validation/opt_kokha_aer_nostandard.dat')
# OR angle, (P11+P12)/2, (P11-P12)/2, P33, P43 (Smartg standard format)
pha=read_phase('/home/did/RTC/SMART-G/validation/opt_kokha_aer_standard.dat', standard=True)

#2) Set single scattering albedo of aerosols to 0.80 for each layer and set the aerosol phase function
aer=AeroOPAC('maritime_clean',0.3262, wl, ssa=0.80, phase=pha)

#3) build profile
atm_custom = AtmAFGL('afglmw', comp=[aer]
                    #4) Could also import aerosol profiles (extinction and ssa) from external files
                    #,prof_aer= (aer_ext_valid,aer_ssa_valid)
                    )

azimuth_transect = 30. 
m = Smartg().run(wl=wl, THVDEG=60., atm=atm_custom, NBPHOTONS=1e8)
_ = smartg_view(m, ind=Idx([azimuth_transect]))

## Multispectral

### Independant spectral computations

In [ ]:
# multispectral simulation
# computation done independently for each wavelegnth, NBPHOTONS is shared equally
# between all wavelegths (thus Monte Carlo NOISE in the spectrum)
# wavelengths is a list or numpy array
wl = np.linspace(555., 565., num=21)

pro = AtmAFGL('afglt',
              pfwav=[560.], # wavelengths for which the phase function is computed
                                     # optional, otherwise phase functions are calculated at all bands
                                     # nearest neighbour is then used during the RT computation
              comp=[AeroOPAC('maritime_clean', 0.3, 550.)])
water = None

le={}
le.update(th = np.array([0.]) *np.pi/180)
le.update(phi= np.array([0.]) *np.pi/180)
m = Smartg().run(wl=wl, le=le,
           THVDEG=40., NBPHOTONS=1e7,
           atm=pro, OUTPUT_LAYERS=3,
           surf=RoughSurface(),
           water=water)

m['I_up (TOA)'].sub()[:,0,0].plot(fmt='o-')

### Profiles reuse

In [ ]:
# once computed, profiles may be re-used
# it could save a lot of time for multi spectral coomputation
m2 = Smartg(double=True).run(wl=wl, le=le,
           THVDEG=40., NBPHOTONS=1e8,
           atm=pro, OUTPUT_LAYERS=0,
           surf=RoughSurface())
m2['I_up (TOA)'].sub()[:,0,0].plot(fmt='o-')

### Correlated spectral computations: ALIS method <br>
Still in development.
Not ready with water simulations

In [ ]:
# The ALIS method is described in Emde et al., 2010, There are some restrictions (see paper) 
# a smartg object has to be created with the alis option as a dictionary.
# it contains one field 'nlow': the numer of wavelengths for which scattering corrections are performed
# it should be equal to -1 for a systematic correction for all wavelengths, or any odd numer for which nlow-1 is a divisor of NW-1
# in any case it should be < 201.
S = Smartg(alis={'nlow':-1}, double=True)

In [ ]:
le={}
le.update(th = np.array([0.]) *np.pi/180)
le.update(phi= np.array([0.]) *np.pi/180)
m3 = S.run(wl=wl, le=le,
           THVDEG=40., NBPHOTONS=1e8,
           atm=pro,OUTPUT_LAYERS=0,
           surf=RoughSurface())
m2['I_up (TOA)'].sub()[:,0,0].plot(fmt='o-').plot(fmt='o-', label='independent')
m3['I_up (TOA)'].sub()[:,0,0].plot(fmt='r-x', label='ALIS', vmin=0.0545, vmax=0.0590)
legend()

## Water profile
Full profile customization

In [ ]:
wl=[390.,450.,550.]
grid=[0., 5., 10.] # in m for water, here sea bottom at 10 m depth
# Seafloor albedo, here grey lambertian reflection with albedo of 0.05
ALB=Albedo_cst(0.05)

#1) Read one phase matrix for particles
phaM=read_phase('/home/did/RTC/SMART-G/validation/opt_hydrosols.dat', kind='oc', standard=True)
#2) Construct a LUT and fill all levels and all wavelengths with the the read phase matrix, You can of course
# fill this LUT with several phase matrices
pha = LUT(phaM.data[None,None,:,:],
          names = ['wav_phase', 'z_phase'] + phaM.names,
          axes = [np.array([
              wl[0]]), np.array([0.])] + phaM.axes
         )
#3) import vertical profiles of pure water absorption and scattering coefficient, 
# particle absorption and scatering coefficient profiles (in m-1) and grid
# with shape (N wavelengths, N levels)
aw=np.array([[0., 0., 0.],[0.0204, 0.0092, 0.0565],[0.0204, 0.0092, 0.0565]]).T 
bw=np.array([[0., 0., 0.],[0.0134, 0.0045, 0.0019],[0.0134, 0.0045, 0.0019]]).T
bp=np.array([[0., 0., 0.],[0.0100, 0.0100, 0.0100],[0.01, 0.01, 0.01]]).T
water_custom = IOP(phase=pha, aw=aw, bw=bw, bp=bp, Z=grid, ALB=ALB)

azimuth_transect = 30. 
m1 = Smartg().run(wl=wl, THVDEG=60.,water=water_custom, NBPHOTONS=1e8, NBPHI=24, NBTHETA=24)
_ = smartg_view(m1, ind=Idx([azimuth_transect]),subdict={'wavelength':Idx(390.)}, Imin=0, Imax=0.1)
_ = smartg_view(m1, ind=Idx([azimuth_transect]),subdict={'wavelength':Idx(450.)}, Imin=0, Imax=0.1)
_ = smartg_view(m1, ind=Idx([azimuth_transect]),subdict={'wavelength':Idx(550.)}, Imin=0, Imax=0.1)

In [ ]:
fig = phase_view(m1,  kind='oc', iw=1)

## Absorption

### Monochromatic absorption

In [ ]:
# standard gaseous absorption with ozone and NO2
# we use the single scattering albedo (ssa) method for computing absorption (BEER=0)
# this is the default method
atm = AtmAFGL('afglms', O3=300., NO2=True
             # optionally import gaseous absorption vertical profile
             #,prof_abs= gas_profile
              )
_ = smartg_view(Smartg().run(THVDEG=60., wl=550., NBPHOTONS=1e8, atm=atm, BEER=0))
# we can also use the equivalent theorem (Beer Lambert law) method for computing absorption (BEER=1)
_ = smartg_view(Smartg().run(THVDEG=60., wl=550., NBPHOTONS=1e8, atm=atm, BEER=1))

### Band gaseous absorption using REPTRAN
<br> REPTRAN is described in Gasteiger et al., 2014 and the data is available at www.libradtran.org <br>
J. Gasteiger, C. Emde, B. Mayer, R. Buras, S.A. Buehler, O. Lemke, Representative wavelengths absorption parameterization applied to satellite channels and spectral bands, Journal of Quantitative Spectroscopy and Radiative Transfer, Volume 148, November 2014, Pages 99-115, ISSN 0022-4073, http://dx.doi.org/10.1016/j.jqsrt.2014.06.024.

#### Example 1: Computation of reflectance in MSG-SEVIRI VIS08 channel

In [ ]:
# REPTRAN k distribution file here MSG/SEVIRI solar channels
SEVIRI_SOLAR = REPTRAN('reptran_solar_msg')

# several ways of selecting bands
# 1) selecting all bands 
ibands = SEVIRI_SOLAR.to_smartg()
# 2) selecting one specific band
ibands = SEVIRI_SOLAR.to_smartg(include='msg1_seviri_ch008')
# 3) selecting all bands that contains "msg1" in the band name
ibands = SEVIRI_SOLAR.to_smartg(include='msg1')
# 4) selecting all bands whose wavelengths of internal bands satisfy the min and max conditions
ibands = SEVIRI_SOLAR.to_smartg(lmax=700.)
# 5) a mix of 3) and 4)
ibands = SEVIRI_SOLAR.to_smartg(include='msg1', lmin=600., lmax=1000.)
# 6) note that lmin and lmax can be lists (they should have the same length)
ibands = SEVIRI_SOLAR.to_smartg(include='msg1', lmin=[600., 1400.], lmax=[700., 2000.])

surf= RoughSurface(SUR=1, WIND=5., NH2O=1.34) 
atm = AtmAFGL('afglms', H2O=4.)

# Run Smart-g for Reptran list of ibands
m1  = Smartg().run(THVDEG=30, wl=ibands.l, NBPHOTONS=1e8, atm=atm, surf=surf, BEER=1, progress=False)

# Postprocessing: regrouping internal bands information into real band (spectral integration)
m1r = reduce_reptran(m1, ibands)

# Plotting bands 
for i,w in enumerate(m1r.axis('wavelength')):
    # we use for that a subset of m1r MLUT
    _ = smartg_view(m1r.sub(d={'wavelength':i}), ind=Idx([0.]))
    print(ibands.get_names()[i])

#### Example 2: Polarized reflectance in O2A band in sun glint view with Local Estimate

In [ ]:
%%time
# k distribution file, here full solar channels at coarse resolution
SOLAR_COARSE = REPTRAN('reptran_solar_coarse')
ibands = SOLAR_COARSE.to_smartg(lmin=757., lmax=770.) # within O2A bands
surf=RoughSurface(SUR=1, WIND=5., NH2O=1.34)
atm=AtmAFGL('afglms', P0=900., comp=[AeroOPAC('continental_average',1.,764.)], pfwav=[764.])
# Evaluate reflectance in specific direction Ths=[60.] and RAA=[180.] using LE
# dict containig directions vectors in radians and coded as float32
le={}
le.update(th =np.array([60.], dtype=np.float32)*np.pi/180)
le.update(phi=np.array([180.],dtype=np.float32)*np.pi/180)

Spp_mult=Smartg(double=False)

In [ ]:
%%time
# Compute first atmosphere profiles at all wavelengths and phase functions for re-use
atmbase=atm.calc(ibands.l)

In [ ]:
%%time
# Several runs: it goes faster as long as atmosphere does not change
for col,th0 in zip(['r','g','b','k'],[0.,30.,60.,75.]):
    # we use the single scattering albedo (ssa) method for computing absorption (BEER=0)
    # this is the default method
    m2_ssa = Spp_mult.run(THVDEG=th0, wl=ibands.l, NBPHOTONS=1e7, BEER=0, atm=atmbase, surf=surf, 
                          le=le, progress=False)
    m2r_ssa = reduce_reptran(m2_ssa, ibands)
    I=m2r_ssa['I_up (TOA)']
    Q=m2r_ssa['Q_up (TOA)']
    U=m2r_ssa['U_up (TOA)']

    LP = (Q*Q + U*U).apply(sqrt, 'LP')
    P = (LP/I*100).apply(abs,'DoLP')
    LP.sub()[:,0,0].plot(fmt='.-'+col,vmin=0, vmax=0.15, label=str(th0))
legend(loc='best')

#### Example 3: Computation of Sentinel3/OLCI spectrum

In [ ]:
SPP=Smartg()

# REPTRAN k distribution file here Sentinel 3 solar channels
SENTINEL_SOLAR = REPTRAN('reptran_solar_sentinel')
ibands = SENTINEL_SOLAR.to_smartg(include='olci') # select OLCI bands

th0=50.
le={}
le.update(th=np.array([th0],dtype=np.float32)*np.pi/180)
le.update(phi=np.array([150.],dtype=np.float32)*np.pi/180)
atm=AtmAFGL('afglms', pfwav=[400.,700.,1000.])

water=IOP_1(.1, DEPTH=50., pfwav=[400.,700.,1000.])
surf=RoughSurface()

# full simulation
m_wsa=reduce_reptran(SPP.run(THVDEG=th0, wl=ibands.l,  le=le, atm=atm , surf=surf, water=water, BEER=1,
             progress=True, OUTPUT_LAYERS=3, NBPHOTONS=1e7,), ibands)

# just water
m_w= reduce_reptran(SPP.run(THVDEG=th0, wl=ibands.l,  le=le,  atm=None, surf=None, water=water, BEER=1,
             progress=True, OUTPUT_LAYERS=3, NBPHOTONS=1e7,), ibands)

# just surface
m_s= reduce_reptran(SPP.run(THVDEG=th0, wl=ibands.l,  le=le,  atm=None, surf=surf, water=None,  BEER=1,
             progress=True, OUTPUT_LAYERS=3, NBPHOTONS=1e7,), ibands)

# no atmosphere
m_ws=reduce_reptran(SPP.run(THVDEG=th0, wl=ibands.l,  le=le,  atm=None, surf=surf, water=water, BEER=1,
             progress=True, OUTPUT_LAYERS=3, NBPHOTONS=1e7,), ibands)

* plots in the perspective of atmospheric corrections

In [ ]:
m_ws['I_up (0-)'].sub()[:,0,0].plot( fmt='c^-', label=r'$0^-  \uparrow$, no atm.')
m_ws['I_up (0+)'].sub()[:,0,0].plot( fmt='c.-', label=r'$0^+  \uparrow$, no atm.')

m_w['I_up (0-)'].sub()[:,0,0].plot( fmt='g^-', label=r'$0^-  \uparrow$, just water')

m_s['I_up (0+)'].sub()[:,0,0].plot( fmt='r.-', label=r'$0^+  \uparrow$, just surf',)

m_wsa['I_up (0-)'].sub()[:,0,0].plot( fmt='k^-', label=r'$0^-  \uparrow$')
m_wsa['I_up (0+)'].sub()[:,0,0].plot( fmt='k.-', label=r'$0^+  \uparrow$')
m_wsa['I_up (TOA)'].sub()[:,0,0].plot(fmt='kx-', label=r'$TOA \uparrow$', vmax=0.2,vmin=0)
legend(loc='best')

#### S2-MSI spectral response functions

In [ ]:
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline
SRF = pd.read_excel('/home/did/RTC/SMART-G/validation/Sentinel-2A MSI Spectral Responses.xlsx',
                   sheetname='Spectral Responses')
w=np.array(SRF['SR_WL'])
winf=[]
wsup=[]
wmedian=[]
w_l=np.array([])
srf_l=np.array([])

for b,srf in enumerate(SRF):
    y=np.array(SRF[srf])
    if b!=0 :
        # edge detection for Filter wavelength boundaries
        # Get a function that evaluates the linear spline at any x
        f = InterpolatedUnivariateSpline(w, y, k=1)
        # Get a function that evaluates the derivative of the linear spline at any x
        dfdw =  f.derivative()
        dydw =  dfdw(w)

        ok=np.where(abs(dydw)>0.0001) # detect edges for SRFs with a threshold on first derivative
        plot(w[ok],y[ok])
        #store boundaries of filter for further reptran, add a 2 nm margin
        w0=w[ok[0][0]]-2
        w1=w[ok[0][-1]]+2
        winf.append(w0)
        wsup.append(w1)
        wmedian.append((w0+w1)/2.)
print(SRF.columns.tolist()[1:])
print(winf,wsup)

In [ ]:
SOLAR_COARSE = REPTRAN('reptran_solar_coarse')
ibands = SOLAR_COARSE.to_smartg()
ibands = SOLAR_COARSE.to_smartg(lmin=winf, lmax=wsup)
atm=AtmAFGL('afglms', pfwav=[400., 800., 1200., 1600., 2200.])
surf=RoughSurface()
le={}
th = np.linspace(0., 60.,  num=6, dtype=np.float32)
phi= np.linspace(0., 180., num=9, dtype=np.float32)
le.update(th =th *np.pi/180)
le.update(phi=phi*np.pi/180)
Spp_mult=Smartg(double=False)

In [ ]:
%%time
# Compute first atmospheric profiles at all wavelengths and phase functions for re-use
atmbase=atm.calc(ibands.l)

In [ ]:
%%time
# Run Smart-g for Reptran list of ibands and reduce
m  = reduce_reptran(Spp_mult.run(THVDEG=60, wl=ibands.l, NBPHOTONS=1e6, le=le, surf=surf, BEER=1, atm=atmbase), ibands)

In [ ]:
# final multiplication with SRFs
print('--------------------------------------')
print('REPTRAN SPECTRUM OUTPUT')
print('--------------------------------------')
m.describe()
wr=m.axis('wavelength') # reptran wavelength grid
MLUTS=[]
for b,srf in enumerate(SRF.columns.tolist()[1:]):
    y=np.array(SRF[srf])
    # interpolate SRFs to reptran wavelengths
    f = InterpolatedUnivariateSpline(w, y, k=1)
    fr= LUT(f(wr), axes=[wr], names=['wavelength'])
    ml = MLUT()
    for l in m:
        for pref in ['I_','Q_','U_','V_','transmission'] :
            if pref in l.desc:
                norm= fr.reduce(np.mean,'wavelength')
                lr = (l * fr).reduce(np.mean,'wavelength', as_lut=True)/norm
                ml.add_lut(lr, desc=l.desc )
    ml.set_attrs(m.attrs)
    ml.set_attr('median wavelength', wmedian[b])
    ml.set_attr('band', b+1)

    MLUTS.append(ml)
ms2 = merge(MLUTS, ['median wavelength'])
ms2.add_axis('band', np.arange(len(wmedian))+1)
print('')
print('--------------------------------------')
print('S2 CHANNELS OUTPUT')
print('--------------------------------------')
ms2.describe()


m['I_up (TOA)'].sub()[:,Idx(160),Idx(50)].plot(fmt='.',vmin=0, vmax=0.3, label=r'$TOA^\uparrow$ : Reptran coarse')
ms2['I_up (TOA)'].sub()[:,Idx(160),Idx(50)].plot(fmt='r^-', vmax=0.3, label=r'$TOA^\uparrow : S2$')
legend()

In [ ]:
for i,(b,col) in enumerate(zip(wmedian, ['r','b','g','c']*2)):
    if i==0:
        fig=transect_view(ms2.sub(d={'median wavelength':i}), ind=Idx(180), logI=True, color='k')    
    _=transect_view(ms2.sub(d={'median wavelength':i}), ind=Idx(180), logI=True, fig=fig, color=col)

### High Spectral Resolution gaseous absorption using HITRAN
<br> HITRAN online API interface (HAPI) in python is used. Sources are availabe at http://hitran.org/hapi/.
<br>R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski, HITRAN Application Programming Interface (HAPI): A comprehensive approach to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
<br> It is recommended to use ALIS to avoid spectral MC noise

#### Example in O2A band

In [ ]:
%%time

# get HAPI 
#import hapi
# alternatively use hapi_gpu module which include a GPU accelerated Voigt Profile routine
import hapi_gpu as hapi

# Initialize Local spectroscopic database
hapi.db_begin('data')

# Set spectral interval in wavelength space
dl= 0.01 # resolution (nm)
lmin=759.
lmax=769.001
NW=int((lmax-lmin)/dl)+1


# convert into the wavenumber space for Hitran
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
print('NW: ', NW, 'dv: ', dv, ' cm-1')

# Download trough internet the required spectroscopic data if necessary
if False:
    hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
    hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
    hapi.fetch('O2i3',7,3,vmin-100,vmax+100)

# Set the vertical grid
NL=21 # number of levels
znew =np.linspace(100.,0.,num=NL)

# define the atmosphere for absorption coefficient computation
atm=AtmAFGL('afglmw', grid=znew, O3=0., NO2=False, pfwav=[765.])

# absorption optical thickness profile array
ao2=np.zeros((NW,NL),dtype=float)

i=0
# loop on each vertical level
for p,t,o2,z in zip(atm.prof.P,atm.prof.T,atm.prof.dens_o2,atm.prof.z):

    # get absorption coefficient and wavenumber for each level
    #nuo2,coefo2 = hapi.absorptionCoefficient_Voigt(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=True,
    #    OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
    #    Environment={'p':p/1013.,'T':t})
    
    # alternatively use GPU accelerated Voigt Profile
    nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=True,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
    
    ao2[:,i] = coefo2 * o2 * 1e5
    i=i+1

#convert the wavenumber grid (cm-1) into a wavelength grid (in nm)
wl=1e7/nuo2[::-1]  

# Add absorption to the atm
atm=AtmAFGL('afglmw', grid=znew, O3=0., NO2=False, pfwav=[765.], prof_abs=ao2[::-1,:])
# Compute profiles for further use
pro = atm.calc(wl)

# build a Smartg object with ALIS
S3=Smartg(alis={'nlow':3}, double=True)

thv = [0.,30.,60., 75.]
le={}
le.update(th=np.array(thv)*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)

In [ ]:
%%time
m3 = S3.run(THVDEG=30., wl=wl, atm=atm, le=le, NBPHOTONS=1e7, progress=True)
m3.attrs['kernel time (s)']

In [ ]:
f=figure()
f.set_size_inches(12,3)
for k,(col,th) in enumerate(zip(['r','g','b','k'],thv)):
    I=m3['I_up (TOA)'].sub()[:,0,k]
    Q=m3['Q_up (TOA)'].sub()[:,0,k]
    U=m3['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='-'+col,vmin=0, label=str(th))

legend(loc='best')

## Irradiances

In [ ]:
Spp=Smartg()
surf=RoughSurface(SUR=3, WIND=10., NH2O=1.34)
atm=AtmAFGL('afglms')
water=IOP_1(chl=1.1, pfwav=[550.], DEPTH=100.) 
wl = np.linspace(400.,700.,num=11)
TH0=75

In [ ]:
# 1) planar and spherical fluxes using the "flux" keyword
# Flux are computed FAST in Cone sampling mode
P = Spp.run(THVDEG=TH0, wl=wl, NBPHOTONS=1e7, atm=atm, surf=surf, water=water,
                 flux='planar', OUTPUT_LAYERS=3)
S = Spp.run(THVDEG=TH0, wl=wl, NBPHOTONS=1e7, atm=atm, surf=surf, water=water, 
                 flux='spherical', OUTPUT_LAYERS=3)

# 2) fluxes could be recomputed from radiances, with care about the directional sampling
M  = Spp.run(THVDEG=TH0, wl=wl, NBPHOTONS=1e7, atm=atm, surf=surf, water=water, OUTPUT_LAYERS=3)

# directional integration for irradiances
Mr = reduce_Irr(M)

In [ ]:
# plot downward fluxes spectra just underwater 
P['flux_down (0-)'].plot(fmt='r.-',vmin=0,vmax=1.5,label='down 0- planar')
S['flux_down (0-)'].plot(fmt='b.-',vmin=0,vmax=1.5,label='down 0- spherical')
Mr['Pflux_down (0-)'].plot(fmt='rx-',vmin=0,vmax=1.5,label='down 0- planar from radiance')
Mr['Sflux_down (0-)'].plot(fmt='bx-',vmin=0,vmax=1.5,label='down 0- spherical from radiance')
legend(loc='best')
# plot upward fluxes spectra just underwater 
figure()
P['flux_up (0-)'].plot(fmt='r.-',vmin=0,vmax=0.05,label='up 0- planar')
S['flux_up (0-)'].plot(fmt='b.-',vmin=0,vmax=0.05,label='up 0- spherical')
Mr['Pflux_up (0-)'].plot(fmt='rx-',vmin=0,vmax=0.05,label='up 0- planar from radiance')
Mr['Sflux_up (0-)'].plot(fmt='bx-',vmin=0,vmax=0.05,label='up 0- spherical from radiance')
legend(loc='best')

# 3) For the particular "Down (0+)" Irradiance, it contains only Diffuse component, the direct component
# is obtained trough the 'direct transmission' field
T = P['direct transmission']
figure()
P['flux_down (0+)'].plot(fmt='r.--',vmin=0,vmax=1.5,label='down 0+ diffuse planar')
(P['flux_down (0+)']+T).plot(fmt='r.-',vmin=0,vmax=1.5,label='down 0+ total planar')
S['flux_down (0+)'].plot(fmt='b.--',vmin=0,vmax=1.5,label='down 0+ diffuse spherical')
(S['flux_down (0+)']+T).plot(fmt='b.-',vmin=0,vmax=1.5,label='down 0+ total spherical')
P['flux_down (0-)'].plot(fmt='rx-',vmin=0,vmax=1.5,label='down 0- planar')
S['flux_down (0-)'].plot(fmt='bx-',vmin=0,vmax=1.5,label='down 0- spherical')
legend(loc='best')


## Looping over parameters

In [ ]:
# loop over some wavelengths
MLUTS = []
Spp=Smartg()
for aot in linspace(0, 1.5, 5):
    M = Spp.run(THVDEG=30.,
               wl=443., NBPHOTONS=1e7,
               atm=AtmAFGL('afglt',comp=[AeroOPAC('desert',aot,443.)]),
               surf=RoughSurface(WIND=5.))
    M.set_attr('AOT', aot) # to define a new axis 'aot' later
    MLUTS.append(M)

In [ ]:
#Merge all previous looped MLUTs into a single one
M = merge(MLUTS, ['AOT'], dtype=float)
M.describe()

In [ ]:
M[0].sub()[:,Idx(90),Idx(45)].plot(fmt='o-')

## Difference between two LUTs

In [ ]:
%%time
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
atm=AtmAFGL('afglt',grid=np.linspace(100.,0.,num=100), O3=400., NO2=False)
ths = np.concatenate((
            np.linspace(0. ,75., num=12, dtype=np.float32),
            np.linspace(76.,89., num=12, dtype=np.float32)))
results = []
le={}
le.update(th_deg = ths)
le.update(phi_deg= np.array([0.], dtype=np.float32))
for pp in [True, False]:
    S=Smartg(pp=pp, alt_move=False, double=True, back=True)
    results.append(S.run(THVDEG=0., wl=550., NBPHOTONS=1e6, le=le, atm=atm, BEER=1, stdev=True))
    
PP = results[0]['I_up (TOA)'].sub()[0,:]
SP = results[1]['I_up (TOA)'].sub()[0,:]

PP_std = results[0]['I_stdev_up (TOA)'].sub()[0,:]
SP_std = results[1]['I_stdev_up (TOA)'].sub()[0,:]

In [ ]:
# show relative difference
figure(figsize=(6,6))
diff = 100.*(SP-PP)/SP
diff.desc = 'Spherical - Plane parallel (%)'
(diff).plot(fmt='ko:', vmin=-2, vmax=10.)
unc = (((SP_std*SP_std + PP_std*PP_std)/2).apply(np.sqrt,r'$\Delta$')/PP)*100.
errorbar(diff.axis('Zenith angles'), diff[:], yerr=(SP_std/SP*200)[:], fmt=None, ecolor='k')

## Interactive simulation

In [ ]:
from IPython.html.widgets.interaction import interact, interact_manual

S=Smartg()
le={}
le={}
le.update(phi=np.array([0.]),dtype=np.float32)
le.update(th=np.array([60.])*np.pi/180,dtype=np.float32)

def simulate(thvdeg, surface, aerosol_model, aot550):
    surf = {True:RoughSurface(), False:None}[surface]
    aer = AeroOPAC(aerosol_model, aot550, 550.)
    print('TOA Intensity : %.5f'%S.run(THVDEG=thvdeg,
             wl=[443.], NBPHOTONS=1e5,le=le,
             atm=AtmAFGL('afglt', comp=[aer]), surf=surf, OUTPUT_LAYERS=3, progress=False
          )['I_up (TOA)'].data)

interact_manual(simulate, thvdeg=(0, 90), surface=True,
                aerosol_model= AeroOPAC('desert', 0.1, 550.).list(), aot550=(0.001, 5., 0.01)
                )